In [1]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from agents.jobs import ScrapedJobDescription, JobSelection

In [2]:

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [7]:
deals = ScrapedJobDescription.fetch(show_progress=True)

 25%|█████████████████████                                                               | 1/4 [00:01<00:03,  1.13s/it]ERROR:agents.jobs:Error parsing job page: 'NoneType' object has no attribute 'get_text'
ERROR:agents.jobs:Error parsing job page: 'NoneType' object has no attribute 'get_text'
ERROR:agents.jobs:Error parsing job page: 'NoneType' object has no attribute 'get_text'
ERROR:agents.jobs:Error parsing job page: 'NoneType' object has no attribute 'get_text'
ERROR:agents.jobs:Error parsing job page: 'NoneType' object has no attribute 'get_text'
ERROR:agents.jobs:Error parsing job page: 'NoneType' object has no attribute 'get_text'
ERROR:agents.jobs:Error parsing job page: 'NoneType' object has no attribute 'get_text'
ERROR:agents.jobs:Error parsing job page: 'NoneType' object has no attribute 'get_text'
ERROR:agents.jobs:Error parsing job page: 'NoneType' object has no attribute 'get_text'
ERROR:agents.jobs:Error parsing job page: 'NoneType' object has no attribute 'get_text'
 

In [8]:
len(deals)

20

In [9]:
jobs=deals

In [11]:
jobs[10].describe()

"Title: Sales Executive\nCompany: G-P\nLocation: Boston, Massachusetts, United States\nSummary: <p>&nbsp;</p><p>At G-P, our mission is to break down barriers to global business, enabling opportunities for everyone, everywhere. With remote-first and diverse teams all around the world, our people are the key to achieving this mission.</p><p>That’s why we trust our Dream Team members with the flexibility and autonomy to do their best and most innovative work, encourage and support their personal growth and career development, and believe in recognition for a job well done.</p><p>The work you’ll be a part of here will have a positive impact on people and their work/life possibilities around the world. Our industry-leading SaaS-based Global Employment Platform™ enables our customers to expand and grow into 180+ countries, creating more opportunities for global success – without requiring entity or subsidiary setup. Beyond the power of our platform, we never forget that behind every hire is 

In [12]:
    SYSTEM_PROMPT = """You identify and summarize the 5 most detailed job descriptions from a list, focusing on the most detailed responsibilities, required skills, and meaningful job titles. Respond strictly in JSON with no explanation, using this format:
    
    {
      "jobs": [
        {
          "job_title": "Extracted job title",
          "responsibilities": "Detailed responsibilities",
          "required_skills": ["Skill1", "Skill2"],
          "prep_questions": ["Question1", "Question2", "Question3"],
          "url": "Job URL"
        },
        ...
      ]
    }
    
    The most important factor is the quality and clarity of responsibilities and skills. Do not include vague or generic jobs. Focus on technical and specific roles with clear descriptions.
    """

In [13]:
    USER_PROMPT_PREFIX = """Respond with the 5 most detailed job descriptions, focusing on:
    - Specific responsibilities (minimum 2 sentences)
    - Clearly listed required skills (minimum 2 skills)
    - Generate 3 interview preparation questions for each job.
    
    Respond strictly in JSON and ensure all jobs include:
    - Job title
    - Responsibilities
    - Required skills
    - Prep questions
    - URL
    
    Job Descriptions:
    """


In [15]:
USER_PROMPT_SUFFIX = "\n\nStrictly respond in JSON with up to 5 jobs."

In [16]:
USER_PROMPT_PREFIX += '\n\n'.join([job.describe() for job in jobs])

In [20]:
print(USER_PROMPT_PREFIX)

Respond with the 5 most detailed job descriptions, focusing on:
- Specific responsibilities (minimum 2 sentences)
- Clearly listed required skills (minimum 2 skills)
- Generate 3 interview preparation questions for each job.

Respond strictly in JSON and ensure all jobs include:
- Job title
- Responsibilities
- Required skills
- Prep questions
- URL

Job Descriptions:
Title: Multifunction App for Performing Arts Lovers
Company: Unknown Company
Location: Unknown Location
Summary: Create an application that combines a comprehensive platform for art dance (including ballet), music and cultural events. The application includes՝  • Profiles of artists and creatives.  • Ticket sales for ballet and other cultural events... (Budget: $250 - $750 USD, Jobs: Android, Graphic Design, iPhone, Logo Design, Mobile App Development)
URL: https://www.freelancer.com/projects/iphone-app-development/Multifunction-App-for-Performing-Arts.html

Title: COM Port Troubleshooting on WinIot C# App
Company: Unknow

In [ ]:
USER_PROMPT_PREFIX

In [17]:
def get_recommendations():
    completion = openai.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": USER_PROMPT_PREFIX}
      ],
        response_format=JobSelection
    )
    result = completion.choices[0].message.parsed
    return result

In [18]:
result = get_recommendations()

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [21]:
result

JobSelection(jobs=[Job(description='The Financial Controller will be responsible for overseeing and managing the company’s financial operations, ensuring accuracy and compliance with accounting standards, regulations, and internal policies. The role requires a strategic thinker with strong analytical skills, a keen eye for detail, and the ability to lead a finance team effectively.', job_title='Financial Controller', company='1840 & Company', location='Quezon City, Metro Manila', responsibilities='Prepare and review monthly, quarterly, and annual financial statements, ensuring timely and accurate submissions to stakeholders. Lead the annual budgeting process, monitor performance, and collaborate with department heads on budget management.', skills=['Financial Reporting', 'Budgeting and Forecasting', 'Leadership', 'Accounting Principles'], url='https://remoteOK.com/remote-jobs/remote-financial-controller-1840-amp-company-1058100', prep_questions=['How do you ensure compliance with accou

In [19]:
print(result)

jobs=[Job(description='The Financial Controller will be responsible for overseeing and managing the company’s financial operations, ensuring accuracy and compliance with accounting standards, regulations, and internal policies. The role requires a strategic thinker with strong analytical skills, a keen eye for detail, and the ability to lead a finance team effectively.', job_title='Financial Controller', company='1840 & Company', location='Quezon City, Metro Manila', responsibilities='Prepare and review monthly, quarterly, and annual financial statements, ensuring timely and accurate submissions to stakeholders. Lead the annual budgeting process, monitor performance, and collaborate with department heads on budget management.', skills=['Financial Reporting', 'Budgeting and Forecasting', 'Leadership', 'Accounting Principles'], url='https://remoteOK.com/remote-jobs/remote-financial-controller-1840-amp-company-1058100', prep_questions=['How do you ensure compliance with accounting standar

In [22]:
result.jobs[0]

Job(description='The Financial Controller will be responsible for overseeing and managing the company’s financial operations, ensuring accuracy and compliance with accounting standards, regulations, and internal policies. The role requires a strategic thinker with strong analytical skills, a keen eye for detail, and the ability to lead a finance team effectively.', job_title='Financial Controller', company='1840 & Company', location='Quezon City, Metro Manila', responsibilities='Prepare and review monthly, quarterly, and annual financial statements, ensuring timely and accurate submissions to stakeholders. Lead the annual budgeting process, monitor performance, and collaborate with department heads on budget management.', skills=['Financial Reporting', 'Budgeting and Forecasting', 'Leadership', 'Accounting Principles'], url='https://remoteOK.com/remote-jobs/remote-financial-controller-1840-amp-company-1058100', prep_questions=['How do you ensure compliance with accounting standards in 

In [3]:
from agents.scanner import ScannerAgent

In [4]:
agent=ScannerAgent()
result=agent.scan()
result

INFO:root:[Job Scanner Agent] Job Scanner Agent is initializing
INFO:root:[Job Scanner Agent] Job Scanner Agent is ready
INFO:root:[Job Scanner Agent] Job Scanner Agent is about to fetch job descriptions
ERROR:agents.jobs:Error parsing job page: 'NoneType' object has no attribute 'get_text'
ERROR:agents.jobs:Error parsing job page: 'NoneType' object has no attribute 'get_text'
ERROR:agents.jobs:Error parsing job page: 'NoneType' object has no attribute 'get_text'
ERROR:agents.jobs:Error parsing job page: 'NoneType' object has no attribute 'get_text'
ERROR:agents.jobs:Error parsing job page: 'NoneType' object has no attribute 'get_text'
ERROR:agents.jobs:Error parsing job page: 'NoneType' object has no attribute 'get_text'
ERROR:agents.jobs:Error parsing job page: 'NoneType' object has no attribute 'get_text'
ERROR:agents.jobs:Error parsing job page: 'NoneType' object has no attribute 'get_text'
ERROR:agents.jobs:Error parsing job page: 'NoneType' object has no attribute 'get_text'
ERRO

JobSelection(jobs=[Job(description='As a Senior Software Engineer (Backend) at Sanity.io, you will be responsible for developing new features and improving the scalability of our content operating system. This includes designing scalable backend architecture, collaborating with cross-functional teams to implement innovative solutions, and conducting code reviews to enhance code quality.', job_title='Senior Software Engineer', company='Sanity', location='Remote', responsibilities='You will architect and maintain the backend services supporting our platforms, ensuring high performance, responsiveness, and reliability. Additionally, you will troubleshoot and optimize performance-related issues while mentoring junior engineers in best coding practices.', skills=['Node.js', 'Microservices', 'Cloud architecture', 'RESTful APIs'], url='https://remoteOK.com/remote-jobs/remote-senior-software-engineer-sanity-1058699', prep_questions=['What techniques do you use to ensure the maintainability of 

In [29]:
import chromadb
DB = "jobs_vectorstore"
client = chromadb.PersistentClient(path=DB)
collection = client.get_or_create_collection('Job_desc')

INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


In [31]:
from agents.frontier_agent import FrontierAgent

In [32]:
agent = FrontierAgent(collection)

[LOG]: Initializing Frontier Agent for job descriptions


INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2


[LOG]: Frontier Agent is ready


In [33]:
res=agent.prepare('''We're looking for an individual who loves challenges and taking on problems with imaginative solutions. Also works well in collaborative teams, and can produce high-quality software under tight constraints. You should be a self-starter, self-motivated, able to work independently, collaborate with multiple multi-functional teams across the globe (US, Singapore, India, and Europe) and work on solutions that have a larger impact on Apple business. You will interact with many other group’s / internal teams at Apple to lead and deliver best-in-class products in an exciting, constantly evolving environment.''')

[LOG]: Performing similarity search in Chroma datastore


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[LOG]: Found similar job descriptions


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


[LOG]: Received response from OpenAI: ```json
{
  "job_title": "iOS Developer",
  "responsibilities": [
    "Collaborate with cross-functi...
[LOG]: Error parsing JSON: Expecting value: line 1 column 1 (char 0)


In [34]:
res

{'job_title': 'iOS Developer',
 'responsibilities': ['Collaborate with cross-functional teams to define, design, and ship new features.',
  'Design and build applications for the iOS platform using Swift/Objective-C.',
  'Continuously evaluate and implement new technologies to maximize development efficiency.',
  'Unit-test code for robustness, including edge cases and usability.',
  'Identify and correct bottlenecks and fix bugs to maintain performance and quality.'],
 'required_skills': ['Proficient in Swift and Objective-C with a good knowledge of their ecosystems.',
  'Solid understanding of object-oriented programming principles.',
  "Experience with Cocoa APIs and Apple's frameworks (Core Data, Core Animation, etc.).",
  'Knowledge of multi-threading, memory management, and performance optimization.',
  'Familiarity with RESTful APIs and integrating with back-end services.',
  'Good sense of UI design and user-oriented focus, following Apple’s design principles.',
  'Proficient u

In [35]:
import json

# Invalid JSON string with unexpected structure


try:
    # Parsing invalid JSON string
    data = json.loads(res)
    print(data)
except json.JSONDecodeError as e:
    print("Error parsing JSON:", e)

TypeError: the JSON object must be str, bytes or bytearray, not dict